In [1]:
import json
import requests
from unittest import TestCase

case = TestCase()

In [2]:
LOCAL = 'http://localhost:8000/'

In [3]:
with open('../tests/testes.json', 'r') as json_in:
    testes = json.load(json_in)
print(testes)

{'desunitizacao': {'operadorregistro': 'string', 'tipodocumentotransporte': 'CE', 'placa': 'string', 'operadorevento': 'string', 'imagens': [{'caminhoarquivo': 'string', 'contentType': 'string', 'datacriacao': '2019-05-31T02:55:50.300000+00:00', 'content': '0', 'datamodificacao': '2019-05-31T02:55:50.300000+00:00'}], 'numero': 'string', 'retificador': False, 'IDEvento': 0, 'dataregistro': '2019-05-28T11:10:34.540Z', 'lotes': [{'qtdevolumes': 0, 'falta': True, 'marca': 'string', 'tipovolume': 'bigbag', 'observacoes': 'string', 'pesolote': 0, 'numerolote': 'string', 'tipodocumentopapel': 'BL', 'tipodocumentodesconsolidacao': 'CE', 'documentodesconsolidacao': 'string', 'documentopapel': 'string', 'qtdefalta': 0, 'acrescimo': True}], 'placasemireboque': 'string', 'documentotransporte': 'string', 'dataevento': '2019-05-28T11:10:34.540Z'}, 'PesagemMaritimo': {'operadorregistro': 'string', 'taraconjunto': 0, 'tipodocumentotransporte': 'CE', 'placa': 'string', 'operadorevento': 'string', 'capt

In [4]:
for classe, teste in testes.items():
    rv = requests.post(LOCAL + classe.lower(), json=teste)
    print(classe)
    print(rv.status_code)
    print(rv.text)

desunitizacao
201
4273874933070458978

PesagemMaritimo
201
5057254149720431223

ocorrencia
201
35540862613471353

pesagemterrestre
201
-734697622934267525

unitizacao
201
-8710139967807684206

DTSC
201
8637110972847485914

pesagemveiculovazio
201
2163621580011596676

AcessoVeiculo
400
"Class 'builtins.dict' is not mapped"

posicaoveiculo
201
604091999014655905

artefatorecinto
201
-4572046313034764957

AvariaLote
201
6411752285493719112

PosicaoConteiner
201
-888358736655736134

operacaonavio
201
7929504720655386351

AcessoPessoa
201
6590807620664223724

posicaolote
201
2466031413226208400

InspecaonaoInvasiva
201
-1114295827052722677



In [5]:
def extractDictAFromB(A, B):
    return dict([(k, B[k]) for k in A.keys() if k in B.keys()])


In [6]:
ltestes = testes.copy()
for classe, teste in ltestes.items():
    print(classe)
    rv = requests.get(LOCAL + classe.lower() + '/' + str(teste['IDEvento']))
    print(rv.status_code)
    if rv.status_code == 200:
        response_json = rv.json()
        for data in ['dataevento', 'dataregistro', 'dataoperacao', 'dataliberacao']:
            if teste.get(data) is not None:
                teste.pop(data)
            if response_json.get(data) is not None:
                response_json.pop(data)
        try:
            sub_response = extractDictAFromB(teste, response_json)
            # case.assertDictEqual(teste, sub_response)
            '''
            for key, value in teste.items():
                if isinstance(value, list):
                    for item in value:
                        response_item =  sub_response.get(key)
                        if response_item is not None:
                            print(value)
                            print(response_item)
                            # case.assertEqual(value, response_item)
            '''
            case.assertDictContainsSubset(teste, sub_response)
            # case.assertDictEqual(lteste, response_json)
        except AssertionError as err:
            print(err)
    # print(rv.text)

PesagemMaritimo
200
unitizacao
200
DTSC
200
posicaoveiculo
200
artefatorecinto
200
AvariaLote
200
PosicaoConteiner
200
InspecaonaoInvasiva
200
desunitizacao
200
ocorrencia
200
pesagemterrestre
200
pesagemveiculovazio
200
AcessoVeiculo
404
operacaonavio
200
AcessoPessoa
200
posicaolote
200


In [7]:
import time
s0 = time.time()
query = {'IDEvento': -1,
       'tipoevento': 'PesagemMaritimo'}
r = requests.get(LOCAL + 'get_eventos_novos',
                  data=query)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 1, 
    "IDEvento": 0, 
    "capturaautomatica": true, 
    "dataevento": "2019-05-28T11:45:43.499000Z", 
    "dataregistro": "2019-05-28T11:45:43.499000Z", 
    "documentotransporte": "string", 
    "hash": 5057254149720431223, 
    "numero": "string", 
    "operadorevento": "string", 
    "operadorregistro": "string", 
    "pesobalanca": 0, 
    "pesobrutodeclarado": 0, 
    "placa": "string", 
    "placasemireboque": "string", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 0, 
    "time_created": "2019-05-31T15:48:53Z", 
    "tipodocumentotransporte": "CE"
  }
]

